In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [2]:
import json,os
from glob import glob
import traceback


def writejson(path,arr,name):
    try:
        os.mkdir(path+'preds/')
    except:
        pass
    f = open(path+'preds/'+name,'w')
    json.dump(arr.tolist(),f)
    f.close()

In [3]:
paths  = ['data/datasets/publiccov_ca/', 'data/datasets/employment_ca/', 'data/datasets/law_school/', 'data/datasets/diabetes/']
cnames = [['AGEP', 'SCHL', 'MAR', 'SEX', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'PINCP', 'ESR', 'FER', 'RAC1P',
       'PUBCOV'],
          ['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR'],
          ['zfygpa', 'zgpa', 'DOB_yr', 'weighted_lsat_ugpa', 'cluster_tier',
       'family_income', 'lsat', 'ugpa', 'isPartTime', 'sex', 'race',
       'pass_bar'],
          ['race', 'sex', 'age', 'admissiontypeid', 'dischargedispositionid',
       'admissionsourceid', 'timeinhospital', 'numlabprocedures',
       'numprocedures', 'nummedications', 'numberoutpatient',
       'numberemergency', 'numberinpatient', 'diag1', 'diag2', 'diag3',
       'numberdiagnoses', 'maxgluserum', 'A1Cresult', 'metformin',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'readmitted']
]

In [5]:
def task(path,cols,num):
    
#     if os.path.exists(path+'preds/baseline_pred.json') == False:
    print(path,num)
    
    ss = ""
    if 'SEX' in cols:
        ss = 'SEX'
    else:
        ss = 'sex'
        
    tgt = cols[-1]

    train_df = pd.read_csv(path+'train.csv',header=None)

    train_df.columns =  cols


    train_df.index = train_df[ss]

    train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    test_df = pd.read_csv(path+'test.csv',header=None)

    test_df.columns =  cols

    test_df.index = test_df[ss]

    test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    


    X_train = train_df.drop([tgt],axis=1)
    X_test = test_df.drop([tgt],axis=1)

    y_train = pd.Series(train_df[tgt])
    y_test = pd.Series(test_df[tgt])

    y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
    y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

    X_merged = pd.concat([X_train,X_test])

    ohe = make_column_transformer(
        (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
        remainder='passthrough', verbose_feature_names_out=False)

    X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

    X_train  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
    X_test = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)
    

    #### BASELINE
    try:
        clf = RandomForestClassifier()
        y_pred_baseline = clf.fit(X_train, y_train).predict(X_test)
        writejson(path,y_pred_baseline,'randomforest_pred'+'_'+str(num)+'.json')
    except:
        traceback.print_exc()
    
    return path,"Done"


In [9]:
s = 'data/datasets/publiccov_ca/'

s.split('/')[-2]

'publiccov_ca'

In [6]:
for path,cols in zip(paths,cnames):
    for i in range(10):
        try:
            task(path,cols,i)
        except:
            traceback.print_exc()

data/datasets/publiccov_ca/ 0
data/datasets/publiccov_ca/ 1
data/datasets/publiccov_ca/ 2
data/datasets/publiccov_ca/ 3
data/datasets/publiccov_ca/ 4
data/datasets/publiccov_ca/ 5
data/datasets/publiccov_ca/ 6
data/datasets/publiccov_ca/ 7
data/datasets/publiccov_ca/ 8
data/datasets/publiccov_ca/ 9
data/datasets/employment_ca/ 0
data/datasets/employment_ca/ 1
data/datasets/employment_ca/ 2
data/datasets/employment_ca/ 3
data/datasets/employment_ca/ 4
data/datasets/employment_ca/ 5
data/datasets/employment_ca/ 6
data/datasets/employment_ca/ 7
data/datasets/employment_ca/ 8
data/datasets/employment_ca/ 9
data/datasets/law_school/ 0
data/datasets/law_school/ 1
data/datasets/law_school/ 2
data/datasets/law_school/ 3
data/datasets/law_school/ 4
data/datasets/law_school/ 5
data/datasets/law_school/ 6
data/datasets/law_school/ 7
data/datasets/law_school/ 8
data/datasets/law_school/ 9
data/datasets/diabetes/ 0
data/datasets/diabetes/ 1
data/datasets/diabetes/ 2
data/datasets/diabetes/ 3
data/d